In [2]:
import numpy as np
import pandas as pd
import cv2
import os, os.path
import matplotlib.pyplot as plt
import seaborn as sns
#from keras.datasets import cifar10
#from google.colab.patches import cv2_imshow

In [3]:
os.listdir("train/")[0:10]

['1.png',
 '10.png',
 '100.png',
 '1000.png',
 '10000.png',
 '10001.png',
 '10002.png',
 '10003.png',
 '10004.png',
 '10005.png']

In [4]:
os.path.splitext(os.listdir("train")[1])

('10', '.png')

In [30]:
#train_df = pd.DataFrame()
#for img in os.listdir("train"):
image1 = cv2.imread("train/1.png")
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2 = cv2.imread("train/2.png")
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#train_df = train_df.append(, ignore_index=True)

In [6]:
cv2.imshow('Original image',image)
cv2.imshow('Gray image', gray_image)
  
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
train_df = pd.DataFrame(columns=range(1024))

In [31]:
grayy1 = gray_image1.reshape(-1)
grayy2 = gray_image2.reshape(-1)

In [34]:
pd.DataFrame(grayy2)

,0
0,171
1,134
2,103
3,101
4,130
...,...
1019,130
1020,134
1021,137
1022,138


In [32]:
train_df.loc[0] = grayy1
train_df.loc[1] = grayy2

In [33]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,61,45,48,57,78,96,113,117,123,126,...,93,96,103,94,72,83,145,189,124,99
1,171,134,103,101,130,164,187,195,152,116,...,23,46,66,91,115,130,134,137,138,137
